In [2]:
import numpy as np
import pandas as p
import requests
from bs4 import BeautifulSoup
import time


# Introdution 

I am a volunteer for the United World Colleges Zimbabwean National Committee(UWZ Zw). This is an NGO whose goal is 
to advertise the UWC movement to High school students in Zimbabwe and process student applications to the different UWC schools. Part of my volunteer work is responding to general enquries from prospective students about UWC. This moivated me to create a chatbot that will trained on UWC data via the LangChain framework. This will automate the responses to FAQs. The chatbot will be hosted on Whatsapp to make it more accessible. 


# Data Collection

### Method: Webscrapping 

The chatbot needs to be trained on UWC content so I decided to scrap all of the content of the "https://www.zw.uwc.org" website. This data should be enough for the chatbot to be able to respond to FAQs. The reason I chose to scrap the data myself instead of using the WebBase loader from the Langchain library was that it wasn't extracting all the information I needed. 


In [3]:
base_url = "https://www.whitman.edu/"
html = requests.get(base_url) #getting the website html file

soup = BeautifulSoup(html.text, 'html') # BeautifulSoup is a library that makes it easy to extract data from an html file
soup

<!DOCTYPE html>
<html class="no-js" lang="en">
<head>
<!-- Google Tag Manager -->
<script>
                (function (w, d, s, l, i) {
                    w[l] = w[l] || []; w[l].push({
                        'gtm.start':
                            new Date().getTime(), event: 'gtm.js'
                    }); var f = d.getElementsByTagName(s)[0],
                        j = d.createElement(s), dl = l != 'dataLayer' ? '&l=' + l : ''; j.async = true; j.src =
                            'https://www.googletagmanager.com/gtm.js?id=' + i + dl; f.parentNode.insertBefore(j, f);
                })(window, document, 'script', 'dataLayer', 'GTM-PN449F');</script>
<!-- End Google Tag Manager -->
<meta content="https://www.whitman.edu/xsd/customsearch4/?q=" name="SearchURL"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<title>Whitman College</title>
<meta content="noodp,noydir" name="robots"/>
<link href="https://

In [4]:
# get all the links on the website. 
links = soup.find_all("a")
for link in links:
    print(link.get("href"))

#main
/
/upward-together
/admission-and-aid/applying-to-whitman
/admission-and-aid/visit-whitman
/alumni
https://give.whitman.edu
/whitman-stories
https://calendar.whitman.edu
http://whitman.cafebonappetit.com/
https://whitman.bncollege.com/
/business-office
/campus-map
/career-prep/career-and-community-engagement-center/career-development/resources-for-job-seekers
/communications
/conferences-events-and-scheduling
/counseling-center
/covid-dashboard
/giving
/human-resources
/institutional-research
/human-resources
https://library.whitman.edu/
/security
/welty-student-health-center
/technology-services
https://whitman.presence.io
/whitman-stories
/alumni
/dean-of-students/care-team
/parents-and-families
/about/inclusive-excellence
/about/president-and-leadership
https://connect.whitman.edu
https://my.whitman.edu
http://whitman.cafebonappetit.com/
https://whitman.bncollege.com/
/business-office
/campus-map
/career-prep/career-and-community-engagement-center/career-development/resources-

In [5]:
# get all the links to the different pages on the website and other hyper links 
other_urls = []
website_pages = []
for l in links:
    
    #ignore the ones that are empty or are 'mailto' links
    if (l.get('href') == None) or (l.get('href') == "") or ('mailto' in l.get('href')):
        pass
    
    # these are the cases where they start with http which means they are hyperlinks to a different website. 
    elif "http" in l.get('href'):
        other_urls.append(l)
    
    # these are the cases where the href starts with "/" which means they are part of the base_url
    elif l.get('href')[0] == "/":
        if len(l.get('href')) >1: 
            website_pages.append(base_url + l.get('href'))

    #these are the cases where the href doesn't start with '/' 
    else:
        website_pages.append(base_url+ "/" + l.get('href'))
        

In [6]:
for link in website_pages:
    print(link)

https://www.whitman.edu//#main
https://www.whitman.edu//upward-together
https://www.whitman.edu//admission-and-aid/applying-to-whitman
https://www.whitman.edu//admission-and-aid/visit-whitman
https://www.whitman.edu//alumni
https://www.whitman.edu//whitman-stories
https://www.whitman.edu//business-office
https://www.whitman.edu//campus-map
https://www.whitman.edu//career-prep/career-and-community-engagement-center/career-development/resources-for-job-seekers
https://www.whitman.edu//communications
https://www.whitman.edu//conferences-events-and-scheduling
https://www.whitman.edu//counseling-center
https://www.whitman.edu//covid-dashboard
https://www.whitman.edu//giving
https://www.whitman.edu//human-resources
https://www.whitman.edu//institutional-research
https://www.whitman.edu//human-resources
https://www.whitman.edu//security
https://www.whitman.edu//welty-student-health-center
https://www.whitman.edu//technology-services
https://www.whitman.edu//whitman-stories
https://www.whitman

In [197]:
print(len(website_pages))
print(len(other_urls))

51
57


In [7]:
#getting all the paragraph tags in the html file of the home page so that we can extract the text content. 
paragraphs = soup.find_all("p")
for p in paragraphs:
    print(p.text)

Due to icy conditions, Whitman College will operate on a two-hour delay.
 
		You’ll Matter Here
	


Nestled at the foot of the Blue Mountains in Walla Walla, Washington, you’ll find a place like no other. Whitman College is a top-tier liberal arts school with accomplished and friendly faculty and passionate and curious students. Here, you can find endless support and opportunities in and outside the classroom—becoming your best self.
From the moment you arrive on our beautiful campus, you’ll belong to a community that challenges you, supports you and wants the best for you. You’ll matter here—and you’ll make a difference.
 
		The natural world undeniably shapes and is shaped by humans. To understand and influence this interaction for the better, you need the intellectual and practical tools to study environmental issues from multiple points of view. The Environmental Studies majors at Whitman College brings science and the humanities together in a flexible, unique academic path that wi

#### From the above output the only relevant information are the middle paragraphs. To clean up the output I am going to remove header and footer information. 

In [201]:
footer = soup.find_all("div", class_ = "footer__e4e__txt")
for foot in footer:
    print(foot.text)

©2024 UWC ZimbabweHigh Visibility Version
  Sitemap
 Terms & conditions Data Protection Policy  Safeguarding
Website Design By  e4education



In [186]:
header = soup.find_all("header")
for head in header:
    print(head.text)





UWC Zimbabwe







Menu



Close



HomeAbout UWCWhat is UWC?UWC Educational Model and PrinciplesUWC History & Founding IdeasAbout the UWC Zimbabwe NCShort CoursesUWC Schools & CollegesSchools & CollegesUWC ExperienceLife at UWCAcademic LifeCreativity, Activity, Service (CAS)School StaffPastoral CareCode of ConductHow to ApplyEligibility CriteriaProcedureAvailable OpportunitiesFAQsSupport UsGiveVolunteerContactContact details








/zw/




UK English










In [166]:
website_pages[0]

'https://www.zw.uwc.org/#content'

In [202]:
 
!rm -rf ./"uwczw-content.txt" #if the file exists, delete before running the code. 

#Extract all the content in the paragraphs tags as shown above for all the pages of the https://www.zw.uwc.org/ website
filename = "uwczw-content.txt"
for i in range(len(website_pages)):
    base_url = website_pages[i]
    
    try:
        session = requests.Session() # enabling cookies so that we access all the content on the webpage. 
        html = session.get(base_url, timeout=60)
        html.raise_for_status()  # Raise an exception for HTTP errors
    except Timeout:
        print(f"Timeout occurred for URL: {base_url}")
        
    except requests.exceptions.RequestException as e:
        print("Error fetching URL:", base_url)
        print("Exception:", e)
        continue  # Continue to the next URL

    soup = BeautifulSoup(html.text, 'html.parser')
    
    #removing all the content in the footer class becuase its not relevant. decompeose() removes the HTML elements from the soup
    footer = soup.find_all("div", class_="footer__e4e__txt")
    header = soup.find_all("header")
    
    for foot in footer:
        if foot:
            foot.decompose()
            
    for head in header:
        if head:
            head.decompose()
    
    # Extract the title of the page
    title = soup.title.text.strip() if soup.title else "Untitled"
    
    paragraphs = soup.find_all("p")

    with open(filename, "a") as uwc:
        uwc.write("\nTitle: " + title + f"\n Reference link: {base_url}\n") #reference link so that the bot can provide the user with relevant links when prompted.
        #uwc.write(page + "\n"*2)
        for p in paragraphs:
            uwc.write(p.text + "\n")  # Add a newline after each paragraph
            
    print(f"Processed URL {i+1}/{len(website_pages)}: {base_url}")
    
    time.sleep(1)
    
uwc.close()
    

Processed URL 1/51: https://www.zw.uwc.org/#content
Processed URL 2/51: https://www.zw.uwc.org/page/?title=About+UWC&pid=3416
Processed URL 3/51: https://www.zw.uwc.org/about
Processed URL 4/51: https://www.zw.uwc.org/educationalmodel
Processed URL 5/51: https://www.zw.uwc.org/history
Processed URL 6/51: https://www.zw.uwc.org/page/?title=About+the+UWC+Zimbabwe+NC&pid=3419
Processed URL 7/51: https://www.zw.uwc.org/page/?title=Short+Courses&pid=3421
Processed URL 8/51: https://www.zw.uwc.org/page/?title=UWC+Schools+%26amp%3B+Colleges&pid=3429
Processed URL 9/51: https://www.zw.uwc.org/schools
Processed URL 10/51: https://www.zw.uwc.org/page/?title=UWC+Experience&pid=3430
Processed URL 11/51: https://www.zw.uwc.org/page/?title=Life+at+UWC&pid=3431
Processed URL 12/51: https://www.zw.uwc.org/academiclife
Processed URL 13/51: https://www.zw.uwc.org/cocurricular
Processed URL 14/51: https://www.zw.uwc.org/collegestaff
Processed URL 15/51: https://www.zw.uwc.org/pastoralcare
Processed URL 1

In [217]:
import pprint
file = open(filename, "r")

In [218]:
file.read()


'\nTitle: UWC Zimbabwe - Home\n Reference link: https://www.zw.uwc.org/#content\n\n\xa0\n\n\n“We’re giving students a formative experience by putting them on a journey of discovery. My 17-year-old self had no clue who I was. The greatest gift we’re giving them...\nWe believe that everyone can play a role in making the world a better place, whichever path they choose.\n\nSo we offer a challenging and broad education that will help you get the qualifications you...\nUnited World Colleges (UWC) is a global movement that makes education a force to unite people, nations and cultures for peace and a sustainable future.\xa0\n\n\n\nAt UWC, our mission is to inspire ...\n\nTitle: UWC Zimbabwe - What is UWC?\n Reference link: https://www.zw.uwc.org/page/?title=About+UWC&pid=3416\n You have not allowed cookies and this content may contain cookies. If you would like to view this content please Allow Cookies Cookie Settings \nYou have not allowed cookies and this content may contain cookies.\nIf yo

In [222]:

# text file to pdf file
from fpdf import FPDF

pdf = FPDF() 

# Add a page
pdf.add_page()

# set style and size of font 
# that you want in the pdf
pdf.set_font("Arial", size = 12)

# open the text file in read mode
f = open(filename, "r")
text = f.read()
text = text.encode('latin-1', 'replace').decode('latin-1')
pdf.multi_cell(0, 10, text)
# # insert the texts in pdf
# for x in f:
# 	pdf.cell(200, 10, txt = x, ln = 1, align = 'C')

# save the pdf with name .pdf
pdf.output("uwc-content.pdf") 


''

# Next Stage: Chatbot Training using LangChain